In [1]:
import os 
import numpy as np 
import json 
import sys 
import tqdm
import trimesh 
sys.path.append("..")
from src.utils import *

In [2]:
# collect all ycb models that has .sdf files
ycb_dir = '/media/junting/SSD_data/ycb'
model_ids = os.listdir(ycb_dir)
metadata = {
    'objects': {},
}
output_dir = '../../data/ycb'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


In [3]:


for model_id in model_ids:
    model_name = model_id[4:]
    model_folder = os.path.join(ycb_dir, model_id)
    sdf_file = os.path.join(model_folder, f'{model_name}.sdf')
    
    if os.path.exists(sdf_file):
        metadata['objects'][model_id] = {
            'model_id': model_id,
            'model_name': model_name
        }

        # calculate the dimension of the object mesh
        # Check if there are Google meshes; else use the TSDF folder
        if "google_16k" in os.listdir(model_folder):
            mesh_type = "google_16k"
        else:
            mesh_type = "tsdf"

        if mesh_type == "google_16k":
            mesh_file = os.path.join(model_folder, "google_16k", "textured.obj")
        elif mesh_type == "tsdf":
            mesh_file = os.path.join(model_folder, "tsdf", "textured.obj")
        mesh = trimesh.load(mesh_file)
        dimension = getMeshDimensions(mesh)
        center = getMeshCenter(mesh)
        metadata['objects'][model_id]['bbox_size'] = dimension.tolist()
        metadata['objects'][model_id]['bbox_center'] = center.tolist()




In [4]:
# filter out objects that are too small, i.e. < 4cm
for model_id in list(metadata['objects'].keys()):
    bbox_size = metadata['objects'][model_id]['bbox_size']
    if np.min(bbox_size) < 0.05:
        del metadata['objects'][model_id]

In [5]:
# manually remove skillet since too difficult to grasp for a gripper
del metadata['objects']['027_skillet']

In [6]:
print(f"Number of objects: {len(metadata['objects'])}")

Number of objects: 27


In [7]:

# save metadata
# with open(os.path.join(output_dir, 'metadata.json'), 'w') as f:
#     json.dump(metadata, f, indent=4)


## Generate thumbnail for each model 

In [19]:
# Use external tool f3d to render thumbnails
for model_id in tqdm.tqdm(metadata['objects'].keys()):
    model_name = metadata['objects'][model_id]['model_name']
    model_folder = os.path.join(ycb_dir, model_id)
    if "google_16k" in os.listdir(model_folder):
        mesh_file = os.path.join(model_folder, "google_16k", "textured.obj")
        material_file = os.path.join(model_folder, "google_16k", "textured_map.png")
    elif "tsdf" in os.listdir(model_folder):
        mesh_file = os.path.join(model_folder, "tsdf", "textured.obj")
        material_file = os.path.join(model_folder, "tsdf", "textured.png")
    
    output_file = os.path.join(output_dir, "object_thumbnails", f"{model_name}.png")

    cmd = f"/usr/bin/f3d --input={mesh_file} --output={output_file} --texture-base-color={material_file}"
    os.system(cmd)

0it [00:00, ?it/s]


## Make collage

In [21]:
# collect all thumbnails in container_thumbnails folder, and collage as one image
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import math

def collage(thumbnail_dir, output_path):

    # Get a list of all the thumbnail file names in the folder
    thumbnail_files = os.listdir(thumbnail_dir)

    # Calculate the number of thumbnails in the folder
    num_thumbnails = len(thumbnail_files)

    # Calculate the size of the square image needed to fit all the thumbnails
    image_size = int(math.ceil(math.sqrt(num_thumbnails)))

    # Create a new blank image to use as the canvas for the collage
    canvas = Image.new('RGB', (image_size * 100, image_size * 100), (255, 255, 255))

    # Loop through each thumbnail file and paste it onto the canvas
    for i in range(num_thumbnails):
        # Open the thumbnail image
        thumbnail = Image.open(os.path.join(thumbnail_dir, thumbnail_files[i]))
        
        # Resize the thumbnail to 100x100 pixels
        thumbnail = thumbnail.resize((100, 100))
        
        # Calculate the x and y coordinates of where to paste the thumbnail onto the canvas
        x = (i % image_size) * 100
        y = (i // image_size) * 100
        
        # Paste the thumbnail onto the canvas
        canvas.paste(thumbnail, (x, y))

    # Save the canvas image
    canvas.save(output_path)

# save object thumbnails as one image
thumbnail_dir = os.path.join(output_dir, "object_thumbnails")
object_collage_path = os.path.join(output_dir, "object_collage.jpg")
collage(thumbnail_dir, object_collage_path)
